# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

# Output File (CSV)
input_data_file = "../Weather/output_data/cities.csv"


city_df = pd.read_csv(input_data_file)
city_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature (F),Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Punta Arenas,-53.15,-70.92,33.80,86,90,6.93,CL,1594660076
1,1,Castro,-24.79,-50.01,73.63,57,2,10.89,BR,1594660076
2,2,Polunochnoye,60.87,60.42,61.54,68,10,3.94,RU,1594660076
3,3,Ushuaia,-54.80,-68.30,33.80,86,90,25.28,AR,1594659857
4,4,Grindavik,63.84,-22.43,53.60,76,75,6.93,IS,1594660076
...,...,...,...,...,...,...,...,...,...,...
543,543,Pishin,30.58,67.00,76.01,49,0,4.61,PK,1594660085
544,544,Ola,59.58,151.28,51.71,85,38,1.45,RU,1594660124
545,545,Laï,9.40,16.30,84.04,64,86,4.56,TD,1594660125
546,546,Tanout,14.97,8.89,90.43,39,28,3.56,NE,1594660125


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Plot Heatmap
gmaps.configure(api_key=g_key)

fig = gmaps.figure()
# Create heat layer

locations = city_df[['Latitude','Longitude']]

weight = city_df['Humidity']

heat_layer = gmaps.heatmap_layer(locations, weights=weight,
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
clean_df = city_df.loc[(city_df['Max Temperature (F)'] > 70 ) & (city_df['Max Temperature (F)'] < 85) & (city_df['Wind Speed'] < 10 ) & (city_df['Cloudiness'] == 0),]
clean_df.dropna(inplace=True)
clean_df
print(clean_df.count())

Unnamed: 0             15
City                   15
Latitude               15
Longitude              15
Max Temperature (F)    15
Humidity               15
Cloudiness             15
Wind Speed             15
Country                15
Date                   15
dtype: int64


C:\Users\Bill\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = clean_df.copy()
hotel_df['Hotel Name'] = 'None'
hotel_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature (F),Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
40,40,Romitan,39.93,64.38,84.20,32,0,5.50,UZ,1594660079,None
88,88,Acajutla,13.59,-89.83,84.20,62,0,3.36,SV,1594660084,None
90,90,Nushki,29.55,66.02,81.57,16,0,2.39,PK,1594659661,None
132,132,Lumeje,-11.55,20.78,71.08,27,0,4.32,AO,1594660088,None
150,150,Forlimpopoli,44.19,12.13,82.00,34,0,5.82,IT,1594660090,None
211,211,Kiryat Gat,31.61,34.76,82.99,64,0,6.35,IL,1594660095,None
226,226,Morro Bay,35.37,-120.85,77.00,88,0,5.99,US,1594660096,None
303,303,Marquette,46.53,-87.63,77.00,52,0,3.00,US,1594660062,None
315,315,Herat Province,34.50,62.00,84.20,23,0,5.82,AF,1594660104,None
368,368,Valparaíso,-33.04,-71.63,71.60,30,0,9.17,CL,1594659732,None


In [6]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "0,0",  # default ,
    "radius" : "5000",
    "type": "restaurant",
    "key": g_key,
}
# use iterrows to iterate through hotel dataframe
for index, row in hotel_df.iterrows():

    # get longitude and latitude from dataframe
    lat = row['Latitude']
    lng = row['Longitude']
    
    params["location"] = f"{lat},{lng}"
    
    temp = params["location"]
    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {temp}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest restaurant is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
       
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 40: 39.93,64.38.
Missing field/result... skipping.
------------
Retrieving Results for Index 88: 13.59,-89.83.
Closest restaurant is Restaurante "El Tayacán".
------------
Retrieving Results for Index 90: 29.55,66.02.
Closest restaurant is Yallanzai cafe.
------------
Retrieving Results for Index 132: -11.55,20.78.
Missing field/result... skipping.
------------
Retrieving Results for Index 150: 44.19,12.13.
Closest restaurant is Locanda alla Mano.
------------
Retrieving Results for Index 211: 31.61,34.76.
Closest restaurant is גבטו- קרית גת.
------------
Retrieving Results for Index 226: 35.37,-120.85.
Closest restaurant is Inn at Morro Bay.
------------
Retrieving Results for Index 303: 46.53,-87.63.
Closest restaurant is Little Caesars Pizza.
------------
Retrieving Results for Index 315: 34.5,62.0.
Missing field/result... skipping.
------------
Retrieving Results for Index 368: -33.04,-71.63.
Closest restaurant is Hotel Mon Reve - SPA - Bar & Restaurant

In [7]:
# show dataframe with Hotel name
hotel_df


,Unnamed: 0,City,Latitude,Longitude,Max Temperature (F),Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
40,40,Romitan,39.93,64.38,84.20,32,0,5.50,UZ,1594660079,None
88,88,Acajutla,13.59,-89.83,84.20,62,0,3.36,SV,1594660084,"Restaurante ""El Tayacán"""
90,90,Nushki,29.55,66.02,81.57,16,0,2.39,PK,1594659661,Yallanzai cafe
132,132,Lumeje,-11.55,20.78,71.08,27,0,4.32,AO,1594660088,None
150,150,Forlimpopoli,44.19,12.13,82.00,34,0,5.82,IT,1594660090,Locanda alla Mano
211,211,Kiryat Gat,31.61,34.76,82.99,64,0,6.35,IL,1594660095,גבטו- קרית גת
226,226,Morro Bay,35.37,-120.85,77.00,88,0,5.99,US,1594660096,Inn at Morro Bay
303,303,Marquette,46.53,-87.63,77.00,52,0,3.00,US,1594660062,Little Caesars Pizza
315,315,Herat Province,34.50,62.00,84.20,23,0,5.82,AF,1594660104,None
368,368,Valparaíso,-33.04,-71.63,71.60,30,0,9.17,CL,1594659732,Hotel Mon Reve - SPA - Bar & Restaurant


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [9]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Display figure
fig.add_layer(marker_layer)

In [10]:
fig


Figure(layout=FigureLayout(height='420px'))